# Alphafold2 on AWS

In [ ]:
import boto3
import os
import json

## 1.Replace the s3 bucket name, api-gateway url and auth key.

In [ ]:
s3_bucket_name = "s3://****"
api_gateway_url = "https://*****.amazonaws.com"
api_gateway_auth_key = "*******"

## 2.Upload fasta file to the input folder

## 3. Upload input folder to s3

In [123]:
!(aws s3 sync ./input/ $s3_bucket_name/input/)

upload: input/.DS_Store to s3://apigwcdkstack-alphafold2s3bucket9e1d428b-xwy4vc6erhav/input/.DS_Store
upload: input/rcsb_pdb_7K59.fasta to s3://apigwcdkstack-alphafold2s3bucket9e1d428b-xwy4vc6erhav/input/rcsb_pdb_7K59.fasta
upload: input/rcsb_pdb_7KCM.fasta to s3://apigwcdkstack-alphafold2s3bucket9e1d428b-xwy4vc6erhav/input/rcsb_pdb_7KCM.fasta
upload: input/rcsb_pdb_1A9X.fasta to s3://apigwcdkstack-alphafold2s3bucket9e1d428b-xwy4vc6erhav/input/rcsb_pdb_1A9X.fasta
upload: input/rcsb_pdb_3JA9_multimer.fasta to s3://apigwcdkstack-alphafold2s3bucket9e1d428b-xwy4vc6erhav/input/rcsb_pdb_3JA9_multimer.fasta
upload: input/rcsb_pdb_3IYL.fasta to s3://apigwcdkstack-alphafold2s3bucket9e1d428b-xwy4vc6erhav/input/rcsb_pdb_3IYL.fasta
upload: input/rcsb_pdb_7ALC_multimer.fasta to s3://apigwcdkstack-alphafold2s3bucket9e1d428b-xwy4vc6erhav/input/rcsb_pdb_7ALC_multimer.fasta
upload: input/rcsb_pdb_6TBT.fasta to s3://apigwcdkstack-alphafold2s3bucket9e1d428b-xwy4vc6erhav/input/rcsb_pdb_6TBT.fasta
upload: 

# API：
## Get All Jobs info, Do not Change anything

In [131]:
!curl -H Authorization:$api_gateway_auth_key $api_gateway_url | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   652  100   652    0     0   2924      0 --:--:-- --:--:-- --:--:--  2936
[
  {
    "que": "low",
    "db_preset": "reduced_dbs",
    "num_multimer_predictions_per_model": 5,
    "fasta": "6TBT",
    "fasta_seq": "GPDSQIQFTRHASDVLLNLNRLRSRDILTDVVIVVSREQFRAHKTVLMACSGLFYSIFTDQLKRNLSVINLDPEINPEGFNILLDFMYTSRLNLREGNIMAVMATAMYLQMEHVVDTCRKFIKASE",
    "gpu": 1,
    "time": "Mon Apr 25 13:06:35 2022",
    "job_id": "d5ac01de-bd73-4c5a-b2eb-7ef1cbdc3b1a",
    "model_preset": "monomer_casp14",
    "run_relax": "true",
    "max_template_date": "2021-11-01",
    "comment": "BCL6 BTB 结构域与 Apt48 肽复合物的晶体结构",
    "file_name": "rcsb_pdb_6TBT.fasta",
    "id": "75da00ef-da72-417c-ac01-da90edccfa3e",
    "job_status": "RUNNABLE"
  }
]


## Post Job, Change parameters below.

In [154]:
# The name of protein to be predicted.
fasta="6TBT" 

# The name of fasta file which stored in S3's input folder.
file_name="rcsb_pdb_6TBT.fasta"

# reduced_dbs: This preset is optimized for speed and lower hardware requirements. It runs with a reduced version of the BFD database. It requires 8 CPU cores (vCPUs), 8 GB of RAM, and 600 GB of disk space.
# full_dbs: This runs with all genetic databases used at CASP14.

db_preset="reduced_dbs"

# monomer:This is the original model used at CASP14 with no ensembling.
# monomer_casp14: This is the original model used at CASP14 with num_ensemble=8, matching our CASP14 configuration. This is largely provided for reproducibility as it is 8x more computationally expensive for limited accuracy gain (+0.1 average GDT gain on CASP14 domains).
# monomer_ptm: This is the original CASP14 model fine tuned with the pTM head, providing a pairwise confidence measure. It is slightly less accurate than the normal monomer model.
# multimer: This is the AlphaFold-Multimer (https://github.com/deepmind/alphafold#citing-this-work) model. To use this model, provide a multi-sequence FASTA file. In addition, the UniProt database should have been downloaded.
model_preset="monomer_casp14"

# If you are predicting the structure of a protein that is already in PDB and you wish to avoid using it as a template, then max_template_date must be set to be before the release date of the structure
max_template_date="2021-11-01"

# The GPU instance to use,which indicated for [p3.2xlarge、p3.8xlarge,p3.16xlarge and p4d.24xlarge].p4 needs manually uncomment.
que="low"

# The number of GPU to use.When using p3 instances，each gpu means 8vcpu，60G mem,when using p4 instance，each gpu means 12vcpu，160G m
gpu=1

comment="BCL6 BTB 结构域与 Apt48 肽复合物的晶体结构"

Do not change anything below, just run~

In [155]:
parameters = [
    {
    "fasta":fasta,
    "file_name":file_name,
    "db_preset":db_preset,
    "model_preset":model_preset,
    "max_template_date":max_template_date,
    "que":que,
    "comment":comment,
    "gpu":gpu
    }
]
json_parameters = json.dumps(parameters)
with open("command.json","w") as file:
    file.write(str(json_parameters))
!cat command.json | jq

[
  {
    "fasta": "6TBT",
    "file_name": "rcsb_pdb_6TBT.fasta",
    "db_preset": "reduced_dbs",
    "model_preset": "monomer_casp14",
    "max_template_date": "2021-11-01",
    "que": "low",
    "comment": "BCL6 BTB 结构域与 Apt48 肽复合物的晶体结构",
    "gpu": 1
  }
]


In [156]:
!curl $api_gateway_url -X "POST" -H Authorization:$api_gateway_auth_key -H "Content-Type: application/json" -d @command.json

body check pass~.

Successful submit Batch job for fasta:6TBT, id:2f5e0a24-c3b9-4a00-85d2-7e33f72f9e12

Please use $(curl $auth-info $api-gateway-url/2f5e0a24-c3b9-4a00-85d2-7e33f72f9e12) to query this job



copy the id above to get/cancel/delete.

## Get One Jobs info, Change the id to your own

In [130]:
id = "75da00ef-da72-417c-ac01-da90edccfa3e"

In [132]:
!curl -H Authorization:$api_gateway_auth_key $api_gateway_url/$id

####

job info : 

{'que': 'low', 'db_preset': 'reduced_dbs', 'num_multimer_predictions_per_model': Decimal('5'), 'fasta': '6TBT', 'fasta_seq': 'GPDSQIQFTRHASDVLLNLNRLRSRDILTDVVIVVSREQFRAHKTVLMACSGLFYSIFTDQLKRNLSVINLDPEINPEGFNILLDFMYTSRLNLREGNIMAVMATAMYLQMEHVVDTCRKFIKASE', 'gpu': Decimal('1'), 'time': 'Mon Apr 25 13:06:35 2022', 'job_id': 'd5ac01de-bd73-4c5a-b2eb-7ef1cbdc3b1a', 'model_preset': 'monomer_casp14', 'run_relax': 'true', 'max_template_date': '2021-11-01', 'comment': 'BCL6 BTB 结构域与 Apt48 肽复合物的晶体结构', 'file_name': 'rcsb_pdb_6TBT.fasta', 'id': '75da00ef-da72-417c-ac01-da90edccfa3e', 'job_status': 'RUNNABLE'}

####

job status :RUNNABLE,going to run.



## Cancel a running job, Change the id to your own

In [136]:
id="75da00ef-da72-417c-ac01-da90edccfa3e"

In [138]:
!curl -X "CANCEL" -H  Authorization:$api_gateway_auth_key $api_gateway_url/$id

Successfully send CANCEL command to Batch job.

####
Please wait 2min and try to query this job again.



## Cancel All jobs, Do not Change anything

In [139]:
!curl -X "CANCEL" -H  Authorization:$api_gateway_auth_key $api_gateway_url

Canceling all running jobs.

Successfully send CANCEL command to Batch job.

####
Please wait 2min and try to query this job again.



## Delete All jobs, Do not Change anything

In [58]:
!curl -X "DELETE" -H  Authorization:$api_gateway_auth_key $api_gateway_url

Deleting all finished and failed jobs.

Successfully send DELETE command to Batch job.

####
Please wait 2min and try to query this job again.



## Delete a finished or failed job, Change the id to your own

In [133]:
id="75da00ef-da72-417c-ac01-da90edccfa3e"

In [134]:
!curl -X "DELETE" -H Authorization:$api_gateway_auth_key $api_gateway_url/$id

You can't DELETE a RUNNABLE job, please use CANCEL method.


## Visualization

In [206]:
file_name_s3 = os.path.splitext(file_name)[0]
output_folder = os.path.join("output/",file_name_s3)
!aws s3 sync $s3_bucket_name/output/$file_name_s3 $output_folder

In [150]:
!pip3 install py3Dmol

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [210]:
import py3Dmol

def View3DMol(self,mol, size=(600, 600), style="stick", surface=False, opacity=0.5, type="mol2"):
    assert style in ('line', 'stick', 'sphere', 'carton')
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(open(mol,'r').read(), type)
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

def StyleSelector(self,mol,size,style):
    return View3DMol(mol,size=(size,size),style=style).show()

def InteractView(mol,size):
    self.StyleSelector, 
    mol=mol,
    size=size,
    style=ipywidgets.Dropdown(
         options=['line', 'stick', 'sphere'],
         value='stick',
         description='Style:')

### Visualization for rank_0

In [220]:
rank_0 = os.path.join(os.getcwd(),output_folder,"ranked_0.pdb")

p_rank_0 = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p_rank_0.addModel(open(rank_0,'r').read(),'pdb')
p_rank_0.setStyle({'cartoon': {'color':'spectrum'}})
p_rank_0.zoomTo()
p_rank_0.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Visualization for relaxed_model_1_pred_0

In [222]:
relaxed_model_1_pred_0 = os.path.join(os.getcwd(),output_folder,"relaxed_model_1_pred_0.pdb")

p_relaxed_model_1_pred_0 = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p_relaxed_model_1_pred_0.addModel(open(relaxed_model_1_pred_0,'r').read(),'pdb')
p_relaxed_model_1_pred_0.setStyle({'cartoon': {'color':'spectrum'}})
p_relaxed_model_1_pred_0.zoomTo()
p_relaxed_model_1_pred_0.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Visualization for unrelaxed_model_1_pred_0

In [223]:
unrelaxed_model_1_pred_0 = os.path.join(os.getcwd(),output_folder,"unrelaxed_model_1_pred_0.pdb")

p_unrelaxed_model_1_pred_0 = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p_unrelaxed_model_1_pred_0.addModel(open(unrelaxed_model_1_pred_0,'r').read(),'pdb')
p_unrelaxed_model_1_pred_0.setStyle({'cartoon': {'color':'spectrum'}})
p_unrelaxed_model_1_pred_0.zoomTo()
p_unrelaxed_model_1_pred_0.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol